In [ ]:
#| default_exp rolling

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from math import sqrt
from typing import Callable, Optional, Tuple

import numpy as np
from numba import njit  # type: ignore

from window_ops.utils import _gt, _lt, _validate_rolling_sizes, first_not_na

In [ ]:
#| hide
import random
from typing import Union

import pandas as pd

/tmp/ipykernel_114728/1690348054.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#| hide
def prepend_nans(
        collection: Union[np.ndarray, pd.Series],
        number_of_nans: int) -> Union[np.ndarray, pd.Series]:
    nans = np.full(number_of_nans, np.nan, dtype=np.float64)
    if isinstance(collection, np.ndarray):
        return np.hstack((nans, collection))
    if isinstance(collection, pd.Series):
        return pd.concat([pd.Series(nans), collection])
    raise ValueError(f'Collection must be np.ndarray or pd.Series, got: {type(collection)}')

np.random.seed(0)
number_of_nans = 10

array = np.random.rand(100)
array_with_nans = prepend_nans(array, number_of_nans)
series = pd.Series(array)
series_with_nans = prepend_nans(series, number_of_nans)
all_nans_array = np.full(100, np.nan)

## Regular

In [ ]:
#| exporti
def _rolling_docstring(*args, **kwargs) -> Callable:
    base_docstring = """Compute the {} over the last non-na window_size samples of the input array starting at min_samples.

    Parameters
    ----------
    input_array : np.ndarray
        Input array
    window_size : int
        Size of the sliding window
    min_samples : int, optional (default=None)
        Minimum number of samples to produce a result, if `None` then it's set to window_size

    Returns
    -------
    np.ndarray
        Array with rolling computation
    """
    def docstring_decorator(function: Callable):
        function.__doc__ = base_docstring.format(function.__name__)
        return function
        
    return docstring_decorator(*args, **kwargs)

In [ ]:
#| hide
def test_rolling_vs_pandas(rolling: Callable,
                           pandas_aggregation: str,
                           lower_bound_for_min_samples: int = 1,
                           non_na_values: int = 5) -> None:
    
    window_size = random.randint(3, 10)
    min_samples = random.randint(2, window_size)
    
    # expanding for [min_samples, window_size), rolling for [window_size, n_samples]
    np.testing.assert_allclose(
        rolling(array, window_size, min_samples=lower_bound_for_min_samples), 
        series.rolling(window_size, min_periods=lower_bound_for_min_samples).agg(pandas_aggregation)
    )

    # arbitrary min_samples and window_size
    np.testing.assert_allclose(
        rolling(array, window_size, min_samples=min_samples), 
        series.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
    )

    # min_samples = window_size
    np.testing.assert_allclose(
        rolling(array, window_size),
        series.rolling(window_size).agg(pandas_aggregation)
    )

    # skip nas
    np.testing.assert_allclose(
        rolling(array_with_nans, window_size, min_samples=min_samples),
        series_with_nans.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
    )

    # |non-na-values| < min_samples
    np.testing.assert_allclose(
        rolling(
            array_with_nans[:number_of_nans + non_na_values],
            window_size=non_na_values+2,
            min_samples=non_na_values+1),
        all_nans_array[:number_of_nans + non_na_values]
    )

    # min_samples < |non-na-values| < window_size
    np.testing.assert_allclose(
        rolling(
            array_with_nans[:number_of_nans + lower_bound_for_min_samples+2], 
            window_size=lower_bound_for_min_samples+1,
            min_samples=lower_bound_for_min_samples),
        np.hstack((
            all_nans_array[:number_of_nans], 
            rolling(
                array_with_nans[number_of_nans : number_of_nans+lower_bound_for_min_samples+2], 
                window_size=lower_bound_for_min_samples+1, 
                min_samples=lower_bound_for_min_samples)
        ))
    )

    # all-nan -> all-nan
    np.testing.assert_equal(
        rolling(all_nans_array, window_size, min_samples=min_samples),
        all_nans_array
    )    

In [ ]:
#| export
@njit
@_rolling_docstring
def rolling_mean(input_array: np.ndarray,
                 window_size: int,
                 min_samples: Optional[int] = None) -> np.ndarray:
    n_samples = input_array.size
    window_size, min_samples = _validate_rolling_sizes(window_size, min_samples)
    
    output_array = np.full_like(input_array, np.nan)
    start_idx = first_not_na(input_array)
    if start_idx + min_samples > n_samples:
        return output_array
    
    accum = 0.
    upper_limit = min(start_idx + window_size, n_samples)
    for i in range(start_idx, upper_limit):
        accum += input_array[i]
        if i + 1 >= start_idx + min_samples:
            output_array[i] = accum / (i - start_idx + 1)
            
    for i in range(start_idx + window_size, n_samples):
        accum += input_array[i] - input_array[i - window_size]
        output_array[i] = accum / window_size

    return output_array

In [ ]:
#| hide
test_rolling_vs_pandas(rolling=rolling_mean, pandas_aggregation='mean')

In [ ]:
#| exporti
@njit
def _rolling_std(input_array: np.ndarray, 
                 window_size: int,
                 min_samples: Optional[int] = None) -> Tuple[np.ndarray, float, float]:
    """Computes the rolling standard deviation using Welford's online algorithm.
    
    Reference: https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_online_algorithm"""
    n_samples = input_array.size
    window_size, min_samples = _validate_rolling_sizes(window_size, min_samples)
    if min_samples < 2:  # type: ignore
        raise ValueError('min_samples must be greater than 1.')

    output_array = np.full_like(input_array, np.nan)
    start_idx = first_not_na(input_array)
    if start_idx + min_samples > n_samples:
        return output_array, 0, 0

    prev_avg = 0.
    curr_avg = input_array[start_idx]
    m2 = 0.
    upper_limit = min(start_idx + window_size, n_samples)
    for i in range(start_idx + 1, upper_limit):
        prev_avg = curr_avg
        curr_avg = prev_avg + (input_array[i] - prev_avg) / (i - start_idx + 1)
        m2 += (input_array[i] - prev_avg) * (input_array[i] - curr_avg)
        m2 = max(m2, 0.0)
        if i + 1 >= start_idx + min_samples:
            output_array[i] = sqrt(m2 / (i - start_idx))
            
    for i in range(start_idx + window_size, n_samples):
        prev_avg = curr_avg
        new_minus_old = input_array[i] - input_array[i-window_size]
        curr_avg = prev_avg + new_minus_old / window_size
        m2 += new_minus_old * (input_array[i] - curr_avg + input_array[i-window_size] - prev_avg)
        m2 = max(m2, 0.0)
        output_array[i] = sqrt(m2 / (window_size - 1))
        
    return output_array, curr_avg, m2

In [ ]:
#| export
@njit
@_rolling_docstring
def rolling_std(input_array: np.ndarray, 
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    output_array, _, _ = _rolling_std(input_array, window_size, min_samples)
    return output_array

In [ ]:
#| hide
test_rolling_vs_pandas(rolling=rolling_std, pandas_aggregation='std', lower_bound_for_min_samples=2)

In [ ]:
#| exporti
@njit 
def _rolling_comp(comp: Callable,
                  input_array: np.ndarray, 
                  window_size: int,
                  min_samples: Optional[int] = None) -> np.ndarray:
    n_samples = input_array.size
    window_size, min_samples = _validate_rolling_sizes(window_size, min_samples)
    
    output_array = np.full_like(input_array, np.nan)
    start_idx = first_not_na(input_array)
    if start_idx + min_samples > n_samples:
        return output_array
    
    upper_limit = min(start_idx + window_size, n_samples)
    pivot = input_array[start_idx]
    for i in range(start_idx, upper_limit):
        if comp(input_array[i], pivot) > 0:
            pivot = input_array[i]
        if i + 1 >= start_idx + min_samples:
            output_array[i] = pivot
    
    for i in range(start_idx + window_size, n_samples):
        pivot = input_array[i]
        for j in range(1, window_size):
            if comp(input_array[i - j], pivot) > 0:
                pivot = input_array[i - j]
        output_array[i] = pivot
    return output_array

In [ ]:
#| export
@njit
@_rolling_docstring
def rolling_max(input_array: np.ndarray,
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    return _rolling_comp(_gt, input_array, window_size, min_samples)

In [ ]:
#| hide
test_rolling_vs_pandas(rolling_max, 'max')

In [ ]:
#| export
@njit
@_rolling_docstring
def rolling_min(x: np.ndarray,
                window_size: int,
                min_samples: Optional[int] = None) -> np.ndarray:
    return _rolling_comp(_lt, x, window_size, min_samples)

In [ ]:
#| hide
test_rolling_vs_pandas(rolling_min, 'min')

In [ ]:
#| export
@njit
def rolling_correlation(x: np.ndarray, window_size: int) -> np.ndarray:
    """Calculates the rolling correlation of a time series.

    Parameters
    ----------
    x : np.ndarray
        Array of time series data.
    window_size : int
        The size of the window to calculate the correlation over.
    
    Returns
    -------
    np.ndarray
        Array with the rolling correlation for each point in time.
    """
    n = len(x)
    result = np.full(n, np.nan)  # Initializes the result with NaNs
    for i in range(window_size, n):
        x1 = x[i - window_size : i]
        x2 = x[i - window_size + 1 : i + 1]
        mean_x1 = np.mean(x1)
        mean_x2 = np.mean(x2)
        std_x1 = np.std(x1)
        std_x2 = np.std(x2)
        if std_x1 == 0.0 or std_x2 == 0.0:
            result[i] = 0.0  # Avoids division by zero
        else:
            cov = np.mean((x1 - mean_x1) * (x2 - mean_x2))
            corr = cov / (std_x1 * std_x2)
            result[i] = corr
    return result

In [ ]:
#| hide
rolling_correlation(np.random.rand(10), 4)

array([        nan,         nan,         nan,         nan, -0.4499269 ,
       -0.34314644, -0.26327872, -0.83073252, -0.2875238 , -0.80489   ])

In [ ]:
#| export
@njit
def rolling_cv(x: np.ndarray, window_size: int) -> np.ndarray:
    """Calculates the rolling coefficient of variation (CV) over a specified window.
    
    Parameters
    ----------
    x : np.ndarray
        Array of time series data.
    window_size : int
        The window size to calculate CV over.
    
    Returns
    -------
    np.ndarray
        An array with the rolling CV for each point in time.
    """
    n = len(x)
    result = np.full(n, 0.0)  # Initializes with 0.0 instead of NaN
    for i in range(window_size - 1, n):
        window_data = x[i - window_size + 1:i + 1]
        sum_data = 0.0
        sum_squares = 0.0
        for val in window_data:
            sum_data += val
            sum_squares += val * val
        mean = sum_data / window_size if window_size > 0 else 0.0
        if mean == 0.0:
            result[i] = 0.0  # Avoids division by zero
        else:
            std = np.sqrt(sum_squares / window_size - mean * mean)
            result[i] = std / mean
    return result

In [ ]:
#| hide
rolling_cv(np.random.rand(10), 2)

array([0.        , 0.30867629, 0.09504531, 0.40328597, 0.46467307,
       0.34478299, 0.3793003 , 0.20502956, 0.20537307, 0.12018532])

In [ ]:
#| export
@njit
def rolling_mean_positive_only(x: np.ndarray, window_size: int) -> np.ndarray:
    """
    Calculates the rolling mean considering only positive sales days, ignoring effects of zero demand.
    
    Parameters
    ----------
    x : np.ndarray
        Array of sales data.
    window_size : int
        The window size to calculate the mean over.
    
    Returns
    -------
    np.ndarray
        An array with the rolling mean for each point in time, considering only days with positive sales.
    """
    n = len(x)
    result = np.full(n, 0.0)  # Initializes with 0.0 instead of NaN
    for i in range(window_size - 1, n):
        window_data = x[i - window_size + 1 : i + 1]
        sum_data = 0.0
        count = 0
        for val in window_data:
            if val > 0:
                sum_data += val
                count += 1
        if count > 0:
            result[i] = sum_data / count
        else:
            result[i] = 0.0  # window_size without positive values, mean is 0
    return result

In [ ]:
#| hide
rolling_mean_positive_only(np.arange(10) - 5, 2)

array([0. , 0. , 0. , 0. , 0. , 0. , 1. , 1.5, 2.5, 3.5])

In [ ]:
#| export
@njit
def rolling_kurtosis(x: np.ndarray, window_size: int) -> np.ndarray:
    """Calculates the rolling kurtosis, helping identify the presence of outliers in sales and how data deviates from a normal distribution.
    
    Parameters
    ----------
    x : np.ndarray
        Array of sales data.
    window_size : int
        The window size to calculate kurtosis over.
    
    Returns
    -------
    np.ndarray
        Array with the rolling kurtosis for each point in time.
    """
    n = len(x)
    result = np.full(n, 0.0)  # Initializes with 0.0 instead of NaN
    for i in range(window_size - 1, n):
        window_data = x[i - window_size + 1:i + 1]
        mean = np.mean(window_data)
        std = np.std(window_data)
        if std > 0:
            kurtosis = np.mean((window_data - mean) ** 4) / (std ** 4) - 3
        else:
            kurtosis = 0.0
        result[i] = kurtosis
    return result

In [ ]:
#| hide
rolling_kurtosis(np.random.rand(10), 2)

array([ 0., -2., -2., -2., -2., -2., -2., -2., -2., -2.])

## Seasonal

In [ ]:
#| exporti
def _seasonal_rolling_docstring(*args, **kwargs) -> Callable:
    base_docstring = """Compute the {} over the last non-na window_size samples for each seasonal period of the input array starting at min_samples.

    Parameters
    ----------
    input_array : np.ndarray
        Input array
    season_length : int
        Length of the seasonal period
    window_size : int
        Size of the sliding window
    min_samples : int, optional (default=None)
        Minimum number of samples to produce a result, if `None` then it's set to window_size

    Returns
    -------
    np.ndarray
        Array with rolling computation
    """
    def docstring_decorator(function: Callable):
        function.__doc__ = base_docstring.format(function.__name__)
        return function
        
    return docstring_decorator(*args, **kwargs)

@njit
def _seasonal_rolling_op(rolling_op: Callable,
                         input_array: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray: 
    n_samples = input_array.size
    output_array = np.full_like(input_array, np.nan)
    for season in range(season_length):
        output_array[season::season_length] = rolling_op(input_array[season::season_length], window_size, min_samples)
    return output_array

In [ ]:
#| hide
from functools import partial

In [ ]:
#| hide
def test_seasonal_rolling_vs_pandas(seasonal_rolling: Callable,
                                    pandas_aggregation: str,
                                    lower_bound_for_min_samples: int = 1,
                                    non_na_values: int = 5) -> None:
    
    window_size = random.randint(3, 4)
    min_samples = random.randint(2, window_size)
    
    # expanding for [min_samples, window_size), rolling for [window_size, n_samples]
    np.testing.assert_allclose(
        seasonal_rolling(array, window_size=window_size, min_samples=lower_bound_for_min_samples), 
        grouped_series.transform(
            lambda x: x.rolling(window_size, min_periods=lower_bound_for_min_samples).agg(pandas_aggregation)
        )
    )

    # arbitrary min_samples and window_size
    np.testing.assert_allclose(
        seasonal_rolling(array, window_size=window_size, min_samples=min_samples), 
        grouped_series.transform(
            lambda x: x.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
        )
    )

    # min_samples = window_size
    np.testing.assert_allclose(
        seasonal_rolling(array, window_size=window_size),
        grouped_series.transform(
            lambda x: x.rolling(window_size).agg(pandas_aggregation))
    )

    # skip nas
    np.testing.assert_allclose(
        seasonal_rolling(array_with_nans, window_size=window_size, min_samples=min_samples),
        grouped_series_with_nans.transform(
            lambda x: x.rolling(window_size, min_periods=min_samples).agg(pandas_aggregation)
        )
    )

    # |non-na-values| < min_samples
    np.testing.assert_allclose(
        seasonal_rolling(
            array_with_nans[:number_of_nans + non_na_values],
            window_size=non_na_values+2,
            min_samples=non_na_values+1),
        all_nans_array[:number_of_nans + non_na_values]
    )

    # min_samples < |non-na-values| < window_size
    np.testing.assert_allclose(
        seasonal_rolling(
            array_with_nans[:number_of_nans + lower_bound_for_min_samples + 2*season_length], 
            window_size=lower_bound_for_min_samples + season_length,
            min_samples=lower_bound_for_min_samples),
        np.hstack((
            all_nans_array[:number_of_nans], 
            seasonal_rolling(
                array_with_nans[number_of_nans : number_of_nans + lower_bound_for_min_samples + 2*season_length], 
                window_size=lower_bound_for_min_samples + season_length,
                min_samples=lower_bound_for_min_samples)
        ))
    )

    # all-nan -> all-nan
    np.testing.assert_equal(
        seasonal_rolling(all_nans_array, window_size=window_size, min_samples=min_samples),
        all_nans_array
    )

    
def get_seasons(season_length, n_samples):
    return np.arange(n_samples) % season_length

season_length = 7
grouped_series = series.groupby(get_seasons(season_length, series.size))
grouped_series_with_nans = series_with_nans.groupby(get_seasons(season_length, series_with_nans.size))

In [ ]:
#| export
@njit
@_seasonal_rolling_docstring
def seasonal_rolling_mean(input_array: np.ndarray,
                          season_length: int,
                          window_size: int,
                          min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_mean, input_array, season_length, window_size, min_samples)

In [ ]:
#| hide
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_mean, season_length=season_length), 'mean')

In [ ]:
#| export
@njit
@_seasonal_rolling_docstring
def seasonal_rolling_std(input_array: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_std, input_array, season_length, window_size, min_samples)

In [ ]:
#| hide
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_std, season_length=season_length), 'std', lower_bound_for_min_samples=2)

In [ ]:
#| export
@njit
@_seasonal_rolling_docstring
def seasonal_rolling_max(input_array: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_max, input_array, season_length, window_size, min_samples)

In [ ]:
#| hide
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_max, season_length=season_length), 'max')

In [ ]:
#| export
@njit
@_seasonal_rolling_docstring
def seasonal_rolling_min(x: np.ndarray,
                         season_length: int,
                         window_size: int,
                         min_samples: Optional[int] = None) -> np.ndarray:
    return _seasonal_rolling_op(rolling_min, x, season_length, window_size, min_samples)

In [ ]:
#| hide
test_seasonal_rolling_vs_pandas(partial(seasonal_rolling_min, season_length=season_length), 'min')